<a href="https://colab.research.google.com/github/Yuchen971/DiveIntoDeepLearning/blob/main/Linear_Neural_Networks/MLP%E5%A4%9A%E5%B1%82%E6%84%9F%E7%9F%A5%E6%9C%BA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#!pip install d2l
#!pip install matplotlib==3.0.0

In [ ]:
import torch
from torch import nn
from d2l import torch as d2l

# 手动实现
## 初始化

In [169]:
batch_size = 256
# 784 是拉长的图片, 10 是 10 个类别, 256 是一层hidden layer的node数目
num_inputs, num_outputs, num_hiddens = 784, 10, 256
# nn.parameter()可以用可以不用
W1 = nn.Parameter(torch.randn(
    num_inputs, num_hiddens, requires_grad=True) * 0.01)
b1 = nn.Parameter(torch.zeros(num_hiddens, requires_grad=True))
# 接受隐藏层, 也就是输入是hidden的node, 输出output
W2 = nn.Parameter(torch.randn(
    num_hiddens, num_outputs, requires_grad=True) * 0.01)
b2 = nn.Parameter(torch.zeros(num_outputs, requires_grad=True))
params = [W1, b1, W2, b2]
train_iter, test_iter = d2l.load_data_fashion_mnist(batch_size)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


# 激活函数, 模型, loss fun

In [117]:
def relu(X):
  a = torch.zeros_like(X)
  return torch.max(X, a)

def net(X):
  X = X.reshape((-1, num_inputs))
  H = relu(X@W1 + b1) # @ 矩阵乘法
  return (H@W2 + b2)

# training

In [155]:
def accuracy(y_hat, y):
  if len(y_hat.shape) > 1 and y_hat.shape[1] > 1:
    y_hat = y_hat.argmax(axis=1) # 每行中最大元素的索引来获得预测类别
  # 将y_hat的数据类型转换为与y的数据类型, 一个32, 一个64
  cmp = y_hat.type(y.dtype) == y 
  # 将 TRUE, FALSE -> 0, 1
  return float(cmp.type(y.dtype).sum()) #return 正确的个数

class Accumulator:
  '''对n个变量叠加'''
  def __init__(self, n):
    self.data = [0.0] * n # 初始化n个数
  def add(self, *args):
    # 元素相加
    self.data = [a + float(b) for a, b in zip(self.data, args)]
  def reset(self):
    self.data = [0.0] * len(self.data)
  def __getitem__(self, idx):
    return self.data[idx]

def evaluate_accuracy(net, data_iter):
  if isinstance(net, torch.nn.Module):
    net.eval() # 评估模式
  metric = Accumulator(2) # 初始化累加的两个参数
  with torch.no_grad():
    for X, y in data_iter:
      # y.numel(): return the length (总数)
      metric.add(accuracy(net(X), y), y.numel())
  return metric[0] / metric[1] # 正确的个数除以总数

def train_epoch_ch3(net, train_iter, loss, updater):
  '''训练模型一个迭代周期, updater: optimizer'''
  if isinstance(net, torch.nn.Module):
    net.train()
  # 训练损失总和、训练准确度总和、样本数初始化
  metric = Accumulator(3)
  for X, y in train_iter:
    # 计算梯度
    y_hat = net(X) # 预测值输出: ([256, 10])
    l = loss(y_hat, y) # cross entropy
    if isinstance(updater, torch.optim.Optimizer):
      # 使用PyTorch内置的优化器和损失函数
      updater.zero_grad()
      l.backward()
      updater.step()
    else:
      # 使用定制的优化器和损失函数
      l.sum().backward() # l是向量, 转换为标量, pytorch自动转换
      updater(X.shape[0]) # X.shape 也就是 batch_size, 因为拉长了
    metric.add(float(l.sum()), accuracy(y_hat, y), y.numel())
  # 返回训练损失和训练精度
  return metric[0] / metric [2], metric[1] / metric [2]

def train_ch3(net, train_iter, test_iter, loss, num_epochs, updater):
  for epoch in range(num_epochs):
    train_metrics = train_epoch_ch3(net, train_iter, loss, updater)
    test_acc = evaluate_accuracy(net, test_iter)
    train_loss, train_acc = train_metrics
    print(f'train_loss = {train_loss}, train_acc = {train_acc}, test_acc = {test_acc}')


In [170]:
num_epochs, lr = 10, 0.5
loss = nn.CrossEntropyLoss(reduction='mean')
updater = torch.optim.SGD(params, lr=lr)
train_ch3(net, train_iter, test_iter, loss, num_epochs, updater)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


train_loss = 0.003144455442329248, train_acc = 0.6976833333333333, test_acc = 0.7695
train_loss = 0.0019457713350653648, train_acc = 0.8154166666666667, test_acc = 0.8023
train_loss = 0.0016577785536646844, train_acc = 0.84355, test_acc = 0.8009
train_loss = 0.0015371723512808482, train_acc = 0.85545, test_acc = 0.8505
train_loss = 0.0014393630956610044, train_acc = 0.8643666666666666, test_acc = 0.8168
train_loss = 0.0013766879759728907, train_acc = 0.8702166666666666, test_acc = 0.8276
train_loss = 0.0013114453750352065, train_acc = 0.87605, test_acc = 0.8464
train_loss = 0.0012177885269125303, train_acc = 0.8845333333333333, test_acc = 0.8678
train_loss = 0.0011908627289036909, train_acc = 0.8867666666666667, test_acc = 0.8461
